# Data For All Wells in Utah

In [1]:
import dataretrieval.nwis as nwis
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import folium
import mplleaflet
from selenium import webdriver
from tqdm.notebook import tqdm
from folium.plugins import MarkerCluster

import os as os #file directory handling package
from scipy.stats import linregress #used for linear regression
import scipy.stats as scs
import scipy as sc #scientific python
import statsmodels.api as sm #used for statistical modeling
import seaborn as sns #a fancy plotting package (good for quick and exploratory plotting, not as good for analysis)
import matplotlib.gridspec as gridspec #used to create complex multi panel plots
import geopandas as gpd

In [2]:
df = nwis.get_info(stateCd = 'UT')  
dfsites = df[0]
dfsites

,agency_cd,site_no,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,coord_meth_cd,coord_acy_cd,...,local_time_fg,reliability_cd,gw_file_cd,nat_aqfr_cd,aqfr_cd,aqfr_type_cd,well_depth_va,hole_depth_va,depth_src_cd,project_no
0,USGS,09163527,"WEST BITTER CREEK NEAR HARLEY DOME, UTAH",ST,392035.0,1090848.0,39.343032,-109.147336,M,F,...,Y,NaN,NNNNNNNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USGS,09163530,COLORADO RIVER BELOW COLORADO-UTAH STATE LINE,ST,390518.0,1090601.0,39.088317,-109.100945,M,F,...,Y,NaN,NNNNNNNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USGS,09163550,"HARLEY DOME WASH NEAR HARLEY DOME, UTAH",ST,390925.0,1090840.0,39.156927,-109.145114,M,F,...,Y,NaN,NNNNNNNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USGS,09163675,"COTTONWOOD WASH AT I-70, NEAR CISCO, UTAH",ST,390454.0,1091301.0,39.081652,-109.217615,M,F,...,Y,NaN,NNNNNNNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USGS,09163700,"CISCO WASH NEAR CISCO, UTAH",ST,385750.0,1092010.0,38.963873,-109.336784,M,F,...,Y,NaN,NNNNNNNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32700,USGS,415956112525201,(B-15- 9)28cbc- 1,GW,415956.0,1125252.0,41.998808,-112.881929,M,S,...,Y,C,YNNNNNNN,NaN,100VLFL,NaN,400.0,NaN,O,NaN
32701,USGS,415956112532501,(B-15- 9)29dbc- 1,GW,415956.0,1125325.0,41.998808,-112.891096,M,S,...,Y,C,YNNNNNNN,NaN,100VLFL,NaN,480.0,NaN,O,NaN
32702,USGS,415957112411001,(B-15- 7)30cbc- 1,GW,415957.0,1124110.0,41.999086,-112.686923,M,S,...,Y,C,YNNNNNNN,NaN,NaN,NaN,228.0,NaN,NaN,NaN
32703,USGS,415958111242901,(A-15- 5)32dbb- 1,GW,415958.0,1112429.0,41.999377,-111.408818,M,T,...,Y,U,NYNNNNNN,NaN,NaN,NaN,191.0,191.0,NaN,NaN


In [3]:
#selecting only the groundwater sites
dfsitesGW = dfsites[dfsites["site_tp_cd"] == "GW"]
dfsitesGW

,agency_cd,site_no,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,coord_meth_cd,coord_acy_cd,...,local_time_fg,reliability_cd,gw_file_cd,nat_aqfr_cd,aqfr_cd,aqfr_type_cd,well_depth_va,hole_depth_va,depth_src_cd,project_no
993,USGS,365954110163301,(D-43-15)32dcd- 1,GW,365954.0,1101633.0,36.998334,-110.276521,M,S,...,Y,C,NNNNNNNN,N300COPLTS,310DCLL,U,120.0,120.0,R,474923900
994,USGS,370000113273301,(C-43-14)31ddd- 1,GW,370000.0,1132733.0,36.999983,-113.459952,M,F,...,Y,C,YNNNNYNN,NaN,NaN,NaN,14.0,NaN,NaN,NaN
995,USGS,370001112575901,B-42-06 32BDB,GW,370001.0,1125759.0,37.000262,-112.967159,M,S,...,Y,C,YYYYNNNN,NaN,111ALVM,NaN,84.0,84.0,NaN,KAN
996,USGS,370006111300401,(D-44- 4) 8bac- 1,GW,370006.0,1113004.0,37.001654,-111.501827,M,S,...,Y,C,Y Y,NaN,227GLNC,X,NaN,NaN,NaN,971600203
1000,USGS,370010111302501,(D-44- 4) 7aab- 1,GW,370010.0,1113025.0,37.002765,-111.507661,M,F,...,Y,C,YYNNNYNN,NaN,NaN,NaN,800.0,800.0,D,464900200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32700,USGS,415956112525201,(B-15- 9)28cbc- 1,GW,415956.0,1125252.0,41.998808,-112.881929,M,S,...,Y,C,YNNNNNNN,NaN,100VLFL,NaN,400.0,NaN,O,NaN
32701,USGS,415956112532501,(B-15- 9)29dbc- 1,GW,415956.0,1125325.0,41.998808,-112.891096,M,S,...,Y,C,YNNNNNNN,NaN,100VLFL,NaN,480.0,NaN,O,NaN
32702,USGS,415957112411001,(B-15- 7)30cbc- 1,GW,415957.0,1124110.0,41.999086,-112.686923,M,S,...,Y,C,YNNNNNNN,NaN,NaN,NaN,228.0,NaN,NaN,NaN
32703,USGS,415958111242901,(A-15- 5)32dbb- 1,GW,415958.0,1112429.0,41.999377,-111.408818,M,T,...,Y,U,NYNNNNNN,NaN,NaN,NaN,191.0,191.0,NaN,NaN


In [4]:
# here I am dropping all the dupliats based on site ID. inplace is used to replace the dataframe:
dfsitesGW.drop_duplicates(subset=['site_no'], keep='first', inplace = True)


# in all cases, I am dropping all the sites that have no coordinates
dfsitesGW.dropna(subset=['dec_lat_va', 'dec_long_va'], inplace=True)

dfsitesGW

C:\Users\jihad\AppData\Local\Temp\ipykernel_27244\2988894916.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsitesGW.drop_duplicates(subset=['site_no'], keep='first', inplace = True)
C:\Users\jihad\AppData\Local\Temp\ipykernel_27244\2988894916.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsitesGW.dropna(subset=['dec_lat_va', 'dec_long_va'], inplace=True)


,agency_cd,site_no,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,coord_meth_cd,coord_acy_cd,...,local_time_fg,reliability_cd,gw_file_cd,nat_aqfr_cd,aqfr_cd,aqfr_type_cd,well_depth_va,hole_depth_va,depth_src_cd,project_no
993,USGS,365954110163301,(D-43-15)32dcd- 1,GW,365954.0,1101633.0,36.998334,-110.276521,M,S,...,Y,C,NNNNNNNN,N300COPLTS,310DCLL,U,120.0,120.0,R,474923900
994,USGS,370000113273301,(C-43-14)31ddd- 1,GW,370000.0,1132733.0,36.999983,-113.459952,M,F,...,Y,C,YNNNNYNN,NaN,NaN,NaN,14.0,NaN,NaN,NaN
995,USGS,370001112575901,B-42-06 32BDB,GW,370001.0,1125759.0,37.000262,-112.967159,M,S,...,Y,C,YYYYNNNN,NaN,111ALVM,NaN,84.0,84.0,NaN,KAN
996,USGS,370006111300401,(D-44- 4) 8bac- 1,GW,370006.0,1113004.0,37.001654,-111.501827,M,S,...,Y,C,Y Y,NaN,227GLNC,X,NaN,NaN,NaN,971600203
1000,USGS,370010111302501,(D-44- 4) 7aab- 1,GW,370010.0,1113025.0,37.002765,-111.507661,M,F,...,Y,C,YYNNNYNN,NaN,NaN,NaN,800.0,800.0,D,464900200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32700,USGS,415956112525201,(B-15- 9)28cbc- 1,GW,415956.0,1125252.0,41.998808,-112.881929,M,S,...,Y,C,YNNNNNNN,NaN,100VLFL,NaN,400.0,NaN,O,NaN
32701,USGS,415956112532501,(B-15- 9)29dbc- 1,GW,415956.0,1125325.0,41.998808,-112.891096,M,S,...,Y,C,YNNNNNNN,NaN,100VLFL,NaN,480.0,NaN,O,NaN
32702,USGS,415957112411001,(B-15- 7)30cbc- 1,GW,415957.0,1124110.0,41.999086,-112.686923,M,S,...,Y,C,YNNNNNNN,NaN,NaN,NaN,228.0,NaN,NaN,NaN
32703,USGS,415958111242901,(A-15- 5)32dbb- 1,GW,415958.0,1112429.0,41.999377,-111.408818,M,T,...,Y,U,NYNNNNNN,NaN,NaN,NaN,191.0,191.0,NaN,NaN


In [5]:
#minimizing the data to work with a small dataset
# new_df = dfsitesGW.iloc[:30]
new_df = dfsitesGW
new_df

,agency_cd,site_no,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,coord_meth_cd,coord_acy_cd,...,local_time_fg,reliability_cd,gw_file_cd,nat_aqfr_cd,aqfr_cd,aqfr_type_cd,well_depth_va,hole_depth_va,depth_src_cd,project_no
993,USGS,365954110163301,(D-43-15)32dcd- 1,GW,365954.0,1101633.0,36.998334,-110.276521,M,S,...,Y,C,NNNNNNNN,N300COPLTS,310DCLL,U,120.0,120.0,R,474923900
994,USGS,370000113273301,(C-43-14)31ddd- 1,GW,370000.0,1132733.0,36.999983,-113.459952,M,F,...,Y,C,YNNNNYNN,NaN,NaN,NaN,14.0,NaN,NaN,NaN
995,USGS,370001112575901,B-42-06 32BDB,GW,370001.0,1125759.0,37.000262,-112.967159,M,S,...,Y,C,YYYYNNNN,NaN,111ALVM,NaN,84.0,84.0,NaN,KAN
996,USGS,370006111300401,(D-44- 4) 8bac- 1,GW,370006.0,1113004.0,37.001654,-111.501827,M,S,...,Y,C,Y Y,NaN,227GLNC,X,NaN,NaN,NaN,971600203
1000,USGS,370010111302501,(D-44- 4) 7aab- 1,GW,370010.0,1113025.0,37.002765,-111.507661,M,F,...,Y,C,YYNNNYNN,NaN,NaN,NaN,800.0,800.0,D,464900200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32700,USGS,415956112525201,(B-15- 9)28cbc- 1,GW,415956.0,1125252.0,41.998808,-112.881929,M,S,...,Y,C,YNNNNNNN,NaN,100VLFL,NaN,400.0,NaN,O,NaN
32701,USGS,415956112532501,(B-15- 9)29dbc- 1,GW,415956.0,1125325.0,41.998808,-112.891096,M,S,...,Y,C,YNNNNNNN,NaN,100VLFL,NaN,480.0,NaN,O,NaN
32702,USGS,415957112411001,(B-15- 7)30cbc- 1,GW,415957.0,1124110.0,41.999086,-112.686923,M,S,...,Y,C,YNNNNNNN,NaN,NaN,NaN,228.0,NaN,NaN,NaN
32703,USGS,415958111242901,(A-15- 5)32dbb- 1,GW,415958.0,1112429.0,41.999377,-111.408818,M,T,...,Y,U,NYNNNNNN,NaN,NaN,NaN,191.0,191.0,NaN,NaN


In [6]:
#calling all data from all sites obtained from the new_df
wellsDf = nwis.get_record(sites= new_df['site_no'], service='gwlevels', start = '1800-01-01') # select all  groundwater level data above 1800
wellsDf.head()
# use the tool to reset index from here immediately

ValueError: Request URL too long. Modify your query to use fewer sites. API response reason: Request-URI Too Long. Pseudo-code example of how to split your query: 
 
                    split_list = np.array_split(site_list, n)  # n is number of chunks to divide query into 

                    data_list = []  # list to store chunk results in 

                    # loop through chunks and make requests 

                    for site_list in split_list: 

                        data = nwis.get_record(sites=site_list, service='dv', start=start, end=end) 

                        data_list.append(data)  # append results to list

In [ ]:
# gwWellsDf = wellsDf[['site_tp_cd','lev_va','lev_dt']]
# gwWellsDf = gwWellsDf.reset_index(drop=False)
# gwWellsDf.set_index('site_no', inplace = True)
# gwWellsDf['countno'] = None
# new_df['countno'] = None
# gwWellsDf.dropna(subset=['lev_va'], inplace=True)

# for site_no in gwWellsDf.index.unique():
#     gwWellsSiteDf = gwWellsDf.loc[gwWellsDf.index == site_no]
#     gwWellsSiteDf['lev_dt']=pd.to_datetime(gwWellsSiteDf['lev_dt'])
#     gwWellsSiteDf.set_index('lev_dt', inplace = True )
#     gwWellsSiteDf.dropna(subset=['lev_va'], inplace=True)
#     print(site_no)
#     plt.plot(gwWellsSiteDf.index, gwWellsSiteDf['lev_va'], label=site_no)
#     plt.legend()
#     plt.xlabel('DateTime')
#     plt.ylabel('Depth')
#     plt.title('Time Series Data of wells groundwater level')
#     plt.show()    
    
# #dropping duplicates from the original dataframe gwWellsDf to be able to have a table of number of counts of each site
#     num_obs = gwWellsSiteDf.shape[0]    
#     gwWellsDf.loc[site_no, 'countno'] = num_obs     


In [ ]:
# gwWellsDf = wellsDf[['site_tp_cd','lev_va','lev_dt']]
# gwWellsDf = gwWellsDf.reset_index(drop=False)
# gwWellsDf.set_index('site_no', inplace = True)

# #creating new columns in the original table for start and end dates of recording
# gwWellsDf['startdate'] = None
# gwWellsDf['enddate'] = None

# #define the new columns as datetime
# gwWellsDf['startdate']=pd.to_datetime(gwWellsDf['startdate'])
# gwWellsDf['enddate']=pd.to_datetime(gwWellsDf['lev_dt'])


# #creating new count columns in both tables
# gwWellsDf['countno'] = None
# new_df['countno'] = None
# gwWellsDf.dropna(subset=['lev_va'], inplace=True)

# for site_no in gwWellsDf.index.unique():
#     gwWellsSiteDf = gwWellsDf.loc[gwWellsDf.index == site_no]
#     gwWellsSiteDf['lev_dt']=pd.to_datetime(gwWellsSiteDf['lev_dt'])
#     start_date = gwWellsSiteDf['lev_dt'].min()
#     end_date = gwWellsSiteDf['lev_dt'].max()
#     print(site_no)
#     print(start_date)
#     print(end_date)
#     gwWellsSiteDf.set_index('lev_dt', inplace = True )
#     gwWellsSiteDf.dropna(subset=['lev_va'], inplace=True)

#     plt.plot(gwWellsSiteDf.index, gwWellsSiteDf['lev_va'], label=site_no)
#     plt.legend()
#     plt.xlabel('DateTime')
#     plt.ylabel('Depth')
#     plt.title('Time Series Data of wells groundwater level')
#     plt.show()    
    
# #dropping duplicates from the original dataframe gwWellsDf to be able to have a table of number of counts of each site
#     num_obs = gwWellsSiteDf.shape[0]    
#     gwWellsDf.loc[site_no, 'startdate'] = start_date
#     gwWellsDf.loc[site_no, 'enddate'] = end_date
#     gwWellsDf.loc[site_no, 'countno'] = num_obs  


In [ ]:

# slope, intercept, r_value, p_value, std_err = linregress(df_y[(df_y.time_year < 2020) & (df_y.time_year > 1955)].time_year,df_y[(df_y.time_year < 2020) & (df_y.time_year > 1955)].discharge)

In [ ]:
gwWellsDf = wellsDf[['site_tp_cd','lev_va','lev_dt']]
gwWellsDf = gwWellsDf.reset_index(drop=False)
gwWellsDf.set_index('site_no', inplace = True)



#creating new columns in the original table for start and end dates of recording
gwWellsDf['startdate'] = None
gwWellsDf['enddate'] = None

#creating new columns in the original table for start and end dates of recording
gwWellsDf['slope_value_recent'] = None
gwWellsDf['gwlevel_slope'] = None
gwWellsDf['gwlevel_slope_recent'] = None

#define the new columns as datetime
gwWellsDf['startdate']=pd.to_datetime(gwWellsDf['startdate'])
gwWellsDf['enddate']=pd.to_datetime(gwWellsDf['lev_dt'])


#creating new count columns in both tables: one for recent data above 100 measurements,
#another one for all data above 100 measurements
gwWellsDf['countno'] = None
gwWellsDf['recentcount'] = None
new_df['countno'] = None
new_df['recentcount'] = None
gwWellsDf.dropna(subset=['lev_va'], inplace=True)

for site_no in gwWellsDf.index.unique():
    gwWellsSiteDf = gwWellsDf.loc[gwWellsDf.index == site_no]
    gwWellsSiteDf['lev_dt']=pd.to_datetime(gwWellsSiteDf['lev_dt'])
    start_date = gwWellsSiteDf['lev_dt'].min()
    end_date = gwWellsSiteDf['lev_dt'].max()
    print(site_no)
    print(start_date)
    print(end_date)
    gwWellsSiteDf=gwWellsSiteDf.sort_values(by='lev_dt', ascending=True)
    gwWellsSiteDf.set_index('lev_dt', inplace = True )
    gwWellsSiteDf.dropna(subset=['lev_va'], inplace=True)

    plt.plot(gwWellsSiteDf.index, gwWellsSiteDf['lev_va'], label=site_no)
    plt.legend(framealpha=0)
    plt.xlabel('DateTime')
    plt.ylabel('Depth')
    plt.title('Time Series Data of wells groundwater level')
    plt.show()    

    num_obs = gwWellsSiteDf.shape[0]
    gwWellsDf.loc[site_no, 'startdate'] = start_date
    gwWellsDf.loc[site_no, 'enddate'] = end_date
    gwWellsDf.loc[site_no, 'countno'] = num_obs 
    
    
    
    gwWellsSiteDf = gwWellsSiteDf.loc['2010-01-01':]
    recent_num_obs = gwWellsSiteDf.shape[0]
    gwWellsDf.loc[site_no, 'recentcount'] = recent_num_obs
    if recent_num_obs >= 2:   

        slope, intercept, r_value, p_value, std_err = linregress(np.arange(len(gwWellsSiteDf)), gwWellsSiteDf['lev_va'])
        gwWellsDf.loc[site_no, 'slope_value_recent'] = slope
        
        if slope <0:
            gwWellsDf.loc[site_no, 'gwlevel_slope_recent'] = 'decreasing'
        else:
            gwWellsDf.loc[site_no, 'gwlevel_slope_recent'] = 'increasing'
    
    
#dropping duplicates from the original dataframe gwWellsDf to be able to have a table of number of counts of each site
#     num_obs = gwWellsSiteDf.shape[0]    
#     gwWellsDf.loc[site_no, 'startdate'] = start_date
#     gwWellsDf.loc[site_no, 'enddate'] = end_date
#     gwWellsDf.loc[site_no, 'countno'] = num_obs  

In [ ]:
gwWellsDf

In [ ]:
# site_no='370006111300401'
# gwWellsSiteDf = gwWellsDf.loc[gwWellsDf.index == site_no]
# gwWellsSiteDf['lev_dt']=pd.to_datetime(gwWellsSiteDf['lev_dt'])
# start_date = gwWellsSiteDf['lev_dt'].min()
# end_date = gwWellsSiteDf['lev_dt'].max()
# print(site_no)
# print(start_date)
# print(end_date)

In [ ]:
gwWellsSiteDf

In [ ]:
gwWellsDf = gwWellsDf.reset_index(drop=False)
gwWellsDf.drop_duplicates(subset=['site_no'], keep='first', inplace = True)
gwWellsDf

In [ ]:
new_df

In [ ]:
#merging the 2 tables to obtain only one table with count number
merged_df = pd.merge(new_df, gwWellsDf[['site_no', 'countno','startdate','enddate','recentcount', 'slope_value_recent', 'gwlevel_slope_recent']], on='site_no', how='left')
merged_df

merged_df['countno_x'].fillna(merged_df['countno_y'], inplace=True)
merged_df['countno_x'].fillna(0, inplace=True)

merged_df['recentcount_x'].fillna(merged_df['recentcount_y'], inplace=True)
merged_df['recentcount_x'].fillna(0, inplace=True)



merged_df['recentcount_x'] = merged_df['recentcount_x'].astype(int)

merged_df['countno_x'] = merged_df['countno_x'].astype(int)

merged_df.drop(columns=['countno_y'], inplace=True)
merged_df.rename(columns={'countno_x': 'count'}, inplace=True)



merged_df.drop(columns=['recentcount_y'], inplace=True)
merged_df.rename(columns={'recentcount_x': 'recentcount'}, inplace=True)


merged_df

In [ ]:
merged_df['color']=None
color_mapper = lambda x: 'blue' if x == 'C' else 'red'
merged_df['color'] = merged_df['aqfr_type_cd'].apply(color_mapper)
merged_df

In [ ]:
#subsetting the data of count above 100
df100 = merged_df[merged_df['count'] > 100]
df100


recentdf100 = merged_df[merged_df['recentcount'] > 100]
recentdf100

In [ ]:
# map = folium.Map(location=[39.25, -111.45], zoom_start=8)
# for index, row in df100.iterrows():
#     folium.Marker(location=[row['dec_lat_va'], row['dec_long_va']], 
#                   popup=row['site_no']).add_to(map)
# map

In [ ]:
# # map = folium.Map(location=[39.25, -111.45], zoom_start=8)
# # for index, row in df100.iterrows():
# #     folium.CircleMarker(location=[row['dec_lat_va'], row['dec_long_va']],radius=row['count']*1000000, color='blue', fill=True, fill_color='red')
# # map

# map = folium.Map(location=[39.25, -111.45], zoom_start=8)
# total_rows = len(merged_df)
# # for index, row in df100.iterrows():
# for index, row in tqdm(merged_df.iterrows(), total=total_rows):
#     folium.CircleMarker(location=[row['dec_lat_va'], row['dec_long_va']],radius=row['count']/50, color=row['color'], fill=True, fill_color=row['color'], popup=row['site_no']).add_to(map)


In [ ]:
merged_df

In [ ]:
# dont forget that wells with no data has no NA start and end dates

In [ ]:
import pandas as pd
import geopandas as gpd

# read in well dataframe as a pandas DataFrame
#wells = pd.read_csv('path/to/well/data.csv')

# convert well dataframe to a GeoDataFrame by creating a Point geometry column
wells = gpd.GeoDataFrame(
    merged_df, 
    geometry=gpd.points_from_xy(merged_df.dec_long_va, merged_df.dec_lat_va),
    crs='EPSG:4326' # specify the coordinate reference system of the points
)

# read in landuse shapefile as a GeoDataFrame
landuse = gpd.read_file(r'C:\Users\jihad\Desktop\jihad\hydroteam\Utah_Water_Related_Land_Use\WaterRelatedLandUse.shp')

# specify the column name that contains the landuse information
landuse_column_name = 'Landuse'

# perform a spatial join between the well GeoDataFrame and the landuse GeoDataFrame, using the specified landuse column
joined = gpd.sjoin(wells, landuse[[landuse_column_name, 'geometry']], op='within')

# create a new column in the well dataframe indicating the landuse of the polygon each well belongs to
wells['landuse'] = joined[landuse_column_name]
wells

#wells.to_csv('')


In [ ]:
def landcolor(x):
    if x == 'Agricultural':
        coloruse='green'
    elif x == 'Urban':
        coloruse='blue'
    else:
        coloruse='grey'
    return coloruse
merged_df['color'] = merged_df['landuse'].apply(landcolor)
merged_df

In [ ]:
# merged_df['color']=None 
# color_mapper = lambda x: 'green' if x == 'Agricultural' else 'red'
# merged_df['color'] = merged_df['landuse'].apply(color_mapper)
# merged_df

In [ ]:
map = folium.Map(location=[39.25, -111.45], zoom_start=8)
total_rows = len(merged_df)
for index, row in tqdm(merged_df.iterrows(), total=total_rows):
    folium.CircleMarker(location=[row['dec_lat_va'], row['dec_long_va']],radius=row['count']/50, color=row['color'], fill=True, fill_color=row['color'], popup=row['site_no']).add_to(map)

#map

In [ ]:
# map = folium.Map(location=[39.25, -111.45], zoom_start=8)
# for index, row in merged_df.iterrows():
#     folium.CircleMarker(location=[row['dec_lat_va'], row['dec_long_va']],radius=row['count']/50, color=row['color'], fill=True, fill_color=row['color'], popup=row['site_no']).add_to(map)
# map